In [1]:
import sqlite3
import pandas as pd

In [2]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [3]:
conn = sqlite3.connect('parch-and-posey.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [5]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,7,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,222,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,223,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,583,"CREATE TABLE accounts (\tid integer,\tname bpc..."


Task 1:

Each company in the accounts table wants to create an email address for each primary_poc. The email address should be the first name of the primary_poc . last name primary_poc @ company name .com.

In [21]:
# sqlite version

pd.read_sql_query(sql='''
SELECT SUBSTR(LOWER(primary_poc), 1, INSTR(primary_poc, ' ') -1) || '.' ||
SUBSTR(LOWER(primary_poc), INSTR(primary_poc, ' ') +1, LENGTH(primary_poc) - INSTR(primary_poc, ' ')) || '@' ||
LOWER(name) || '.com' email
FROM accounts;
''', con=conn)

,email
0,tamara.tuma@walmart.com
1,sung.shields@exxon mobil.com
2,jodee.lupo@apple.com
3,serafina.banda@berkshire hathaway.com
4,angeles.crusoe@mckesson.com
...,...
346,buffy.azure@kkr.com
347,esta.engelhardt@oneok.com
348,khadijah.riemann@newmont mining.com
349,deanne.hertlein@ppl.com


Solution:

WITH t1 AS (
 SELECT LEFT(primary_poc,     STRPOS(primary_poc, ' ') -1 ) first_name,  RIGHT(primary_poc, LENGTH(primary_poc) - STRPOS(primary_poc, ' ')) last_name, name
 FROM accounts)
SELECT first_name, last_name, CONCAT(first_name, '.', last_name, '@', name, '.com')
FROM t1;

Task 2:

You may have noticed that in the previous solution some of the company names include spaces, which will certainly not work in an email address. See if you can create an email address that will work by removing all of the spaces in the account name, but otherwise your solution should be just as in question 1. Some helpful documentation is here:
https://www.postgresql.org/docs/8.1/functions-string.html

In [23]:
# sqlite version

pd.read_sql_query(sql='''
SELECT SUBSTR(LOWER(primary_poc), 1, INSTR(primary_poc, ' ') -1) || '.' ||
SUBSTR(LOWER(primary_poc), INSTR(primary_poc, ' ') +1, LENGTH(primary_poc) - INSTR(primary_poc, ' ')) || '@' ||
REPLACE(LOWER(name), ' ', '') || '.com' email
FROM accounts;
''', con=conn)

,email
0,tamara.tuma@walmart.com
1,sung.shields@exxonmobil.com
2,jodee.lupo@apple.com
3,serafina.banda@berkshirehathaway.com
4,angeles.crusoe@mckesson.com
...,...
346,buffy.azure@kkr.com
347,esta.engelhardt@oneok.com
348,khadijah.riemann@newmontmining.com
349,deanne.hertlein@ppl.com


Solution:

WITH t1 AS (
 SELECT LEFT(primary_poc,     STRPOS(primary_poc, ' ') -1 ) first_name,  RIGHT(primary_poc, LENGTH(primary_poc) - STRPOS(primary_poc, ' ')) last_name, name
 FROM accounts)
SELECT first_name, last_name, CONCAT(first_name, '.', last_name, '@', REPLACE(name, ' ', ''), '.com')
FROM  t1;

Task 3:

We would also like to create an initial password, which they will change after their first log in. The first password will be the first letter of the primary_poc's first name (lowercase), then the last letter of their first name (lowercase), the first letter of their last name (lowercase), the last letter of their last name (lowercase), the number of letters in their first name, the number of letters in their last name, and then the name of the company they are working with, all capitalized with no spaces.

In [38]:
# sqlite version

pd.read_sql_query(sql='''
WITH t1 AS (
SELECT SUBSTR(LOWER(primary_poc), 1, INSTR(primary_poc, ' ') -1) first_name,
SUBSTR(LOWER(primary_poc), INSTR(primary_poc, ' ') +1, LENGTH(primary_poc) - INSTR(primary_poc, ' ')) last_name,
REPLACE(UPPER(name), ' ', '') company_name
FROM accounts)

SELECT first_name || '.' || last_name || '@' || company_name || '.com' AS email,
SUBSTR(first_name,1,1) || SUBSTR(first_name,LENGTH(first_name),1) ||
SUBSTR(last_name,1,1) || SUBSTR(last_name,LENGTH(last_name),1) || LENGTH(first_name) || LENGTH(last_name) || company_name
AS password
FROM t1;
''', con=conn)

,email,password
0,tamara.tuma@WALMART.com,tata64WALMART
1,sung.shields@EXXONMOBIL.com,sgss47EXXONMOBIL
2,jodee.lupo@APPLE.com,jelo54APPLE
3,serafina.banda@BERKSHIREHATHAWAY.com,saba85BERKSHIREHATHAWAY
4,angeles.crusoe@MCKESSON.com,asce76MCKESSON
...,...,...
346,buffy.azure@KKR.com,byae55KKR
347,esta.engelhardt@ONEOK.com,eaet410ONEOK
348,khadijah.riemann@NEWMONTMINING.com,khrn87NEWMONTMINING
349,deanne.hertlein@PPL.com,dehn68PPL


Solution:

WITH t1 AS (
 SELECT LEFT(primary_poc,     STRPOS(primary_poc, ' ') -1 ) first_name,  RIGHT(primary_poc, LENGTH(primary_poc) - STRPOS(primary_poc, ' ')) last_name, name
 FROM accounts)
SELECT first_name, last_name, CONCAT(first_name, '.', last_name, '@', name, '.com'), LEFT(LOWER(first_name), 1) || RIGHT(LOWER(first_name), 1) || LEFT(LOWER(last_name), 1) || RIGHT(LOWER(last_name), 1) || LENGTH(first_name) || LENGTH(last_name) || REPLACE(UPPER(name), ' ', '')
FROM t1;